In [19]:
import requests
from bs4 import BeautifulSoup

# Replace spaces in the player's name with '%20' (URL-encoded space)
player_name = "Parvez Rasool"
search_name = player_name.replace(" ", "%20")

# Set headers to mimic a real browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Step 1: Search for the player on ESPN Cricinfo
search_url = f"https://search.espncricinfo.com/ci/content/site/search.html?search={search_name}"
response = requests.get(search_url, headers=headers)

# Step 2: Parse the search results page to find the first player's profile link
soup = BeautifulSoup(response.text, "html.parser")

# Find all links in the search results that point to a cricketer's profile
# These links typically contain "/cricketers/" in the URL
player_links = soup.find_all("a", href=True)
for link in player_links:
    if "/cricketers/" in link["href"]:
        profile_url = f"https://www.espncricinfo.com{link['href']}"
        print(f"Player profile URL: {profile_url}")
        break
else:
    print("No cricketer profile found.")

Player profile URL: https://www.espncricinfo.comhttps://www.espncricinfo.com/cricketers/parvez-rasool-378496


In [23]:
import polars as pl
import requests
from bs4 import BeautifulSoup

# Function to extract player profile URL and ID
def get_player_profile(player_name):
    search_name = player_name.replace(" ", "%20")  # URL-encode the player name
    search_url = f"https://search.espncricinfo.com/ci/content/site/search.html?search={search_name}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the first player profile link
    player_links = soup.find_all("a", href=True)
    for link in player_links:
        if "/cricketers/" in link["href"]:
            profile_url = f"https://www.espncricinfo.com{link['href']}"
            player_id = profile_url.split("-")[-1]  # Extract player ID from URL
            return profile_url, player_id
    return None, None

# Load the CSV with player names into a Polars DataFrame
df = pl.read_csv("player_names.csv")  # Assuming the CSV has a 'Player' column

# Add new columns for profile URL and player ID
df = df.with_columns([
    pl.lit("").alias("profile_url"),
    pl.lit("").alias("player_id")
])

# List to hold the updated data
updated_data = []

# Iterate over the DataFrame and search for each player
for row in df.rows(named=True):
    player_name = row['Player']
    profile_url, player_id = get_player_profile(player_name)
    
    if profile_url and player_id:
        updated_data.append((player_name, profile_url, player_id))
    else:
        updated_data.append((player_name, "Not found", "Not found"))

# Convert the updated data back to a Polars DataFrame
updated_df = pl.DataFrame(updated_data, schema=["Player", "profile_url", "player_id"])

# Save the updated DataFrame back to a CSV file
updated_df.write_csv("players_with_ids.csv")

print("Player profiles and IDs have been added to the CSV.")

Player profiles and IDs have been added to the CSV.


c:\Users\chavv\anaconda\envs\scrappper\lib\functools.py:888: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)
